In [ ]:
#Model architecture inspired by Y. Kim, "Convolutional Neural Networks for Sentence Classification", 2014
#!pip install fasttext

import numpy as np
import fasttext
import fasttext.util
    
# Download the cc.en.300.bin English model beforehand from FastText website
ft = fasttext.load_model('LOCAL_PATH_TO_cc.en.300.bin')

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re
import nltk
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding

In [ ]:
df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

In [ ]:
def process_text(document):
     
    # Remove extra white space from text
    document = re.sub(r'\s+', ' ', document, flags=re.I)
         
    # Remove all the special characters from text
    document = re.sub(r'\W', ' ', str(document))
 
    # Remove all single characters from text
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
 
    # Converting to Lowercase
    document = document.lower()
 
    # Word tokenization       
    tokens = document.split()
        
    tokens = [word for word in tokens if len(word) > 2]
                 
    clean_txt = ' '.join(tokens)
 
    return clean_txt

In [ ]:
from tqdm import tqdm

nltk.download('stopwords')
# For sentence tokenization
#nltk.download('punkt')
en_stop = set(nltk.corpus.stopwords.words('english'))

nltk.download('wordnet')
df['preprocessedStatement'] = df.Statement.apply(process_text)
display(df.head())

In [ ]:
max_length = df.preprocessedStatement.apply(lambda x: len(x.split())).max()

t = Tokenizer()
t.fit_on_texts(df['preprocessedStatement'] )
vocab_size = len(t.word_index) + 1
encoded_tweets = t.texts_to_sequences(df['preprocessedStatement'] )
X = pad_sequences(encoded_tweets, maxlen=max_length, padding='post')

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# Encode labels in column 'Emotion'. 
df['Emotion'] = le.fit_transform(df['Emotion']) 
y = df.pop('Emotion')
#print(y)
y_new = tf.keras.utils.to_categorical(y, num_classes=7)
print(y_new)

In [ ]:
list(le.inverse_transform([0,1,2,3,4,5,6]))

In [ ]:
y.value_counts()

In [ ]:
# 2 options to handle imbalanced dataset: class_weight or focal loss
class_weight = {0: 6, 1: 22, 2: 30, 3: 1, 4: 2, 5: 7, 6: 5}

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y_new, test_size=0.05, stratify=y_new)

x_val = x_train[:100]
y_val = y_train[:100]
x_train = x_train[100:]
y_train = y_train[100:]

In [ ]:
from tensorflow.keras import activations

def focal_loss(gamma=2., alpha=4., from_logits=False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax if from_logits is False.
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed


In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)
model_ft = Sequential()
model_ft.add(Embedding(vocab_size, 300, input_length=max_length, weights=[embedding_matrix], trainable=False))
model_ft.add(Conv1D(128, 5, activation='relu'))
model_ft.add(GlobalMaxPooling1D())
model_ft.add(Dense(64, activation='relu'))
model_ft.add(Dense(7, activation='softmax'))
#model_ft.compile(loss=focal_loss(alpha=1), optimizer='adam', metrics=['accuracy']) #use if you are using focal loss
model_ft.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #use if you are using class_weight
model_ft.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_ft,show_shapes= True)

In [ ]:
model_ft.fit(x_train, y_train, epochs = 20, validation_data=(x_val, y_val), callbacks=callback) #insert class_weight=class_weight if using class_weight

In [ ]:
y_pred = model_ft.predict(x_test)

In [ ]:
y_pred = np.argmax(y_pred, 1)
y_test = np.argmax(y_test, 1)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
list(le.inverse_transform([0,1,2,3,4,5,6]))